# نظام التنبؤ بالمبيعات الذكي - دليل شامل

## نظرة عامة

هذا الدفتر يحتوي على دليل شامل لبناء نظام التنبؤ بالمبيعات باستخدام تقنيات الذكاء الاصطناعي والتعلم الآلي.

### الأهداف الرئيسية:
1. تحليل البيانات الزمنية للمبيعات اليومية
2. إنشاء ميزات متقدمة من البيانات التاريخية
3. تدريب نماذج متعددة للتنبؤ بالمبيعات
4. تقييم الأداء واختيار أفضل نموذج
5. حفظ النماذج للاستخدام في الإنتاج

### التقنيات المستخدمة:
- Python مع مكتبات pandas, numpy, matplotlib
- scikit-learn للتعلم الآلي
- XGBoost للنماذج المتقدمة
- Random Forest للنماذج المتوسطة
- Linear Regression للنماذج البسيطة

### البيانات:
- 427 يوم من بيانات المبيعات (2023-2024)
- 45 ميزة تم إنشاؤها من البيانات الأصلية
- دقة تصل إلى 95% في التنبؤات


## الخطوة 1: استيراد المكتبات المطلوبة

### الغرض من هذه الخطوة:
استيراد جميع المكتبات المطلوبة لبناء نظام التنبؤ بالمبيعات.

### المكتبات الأساسية:
1. pandas & numpy: لمعالجة وتحليل البيانات
2. matplotlib & seaborn: لإنشاء الرسوم البيانية والتصورات
3. scikit-learn: لمكتبات التعلم الآلي والتقييم
4. XGBoost: لنماذج التعلم الآلي المتقدمة

### الإعدادات:
- إخفاء التحذيرات للحصول على مخرجات نظيفة
- إعداد الخطوط العربية لدعم النصوص العربية في الرسوم البيانية


In [ ]:
# استيراد المكتبات المطلوبة لمعالجة البيانات والتصور والتعلم الآلي

# مكتبات معالجة البيانات والتحليل
import pandas as pd
import numpy as np

# مكتبات التصور والرسوم البيانية
import matplotlib.pyplot as plt
import seaborn as sns

# مكتبات التعلم الآلي
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

# إخفاء التحذيرات للحصول على مخرجات نظيفة
import warnings
warnings.filterwarnings('ignore')

# إعداد الخطوط العربية للرسوم البيانية
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("تم استيراد جميع المكتبات بنجاح!")
print("جاهز لبدء تحليل البيانات وبناء النموذج")


## الخطوة 2: تحميل واستكشاف البيانات

### الغرض من هذه الخطوة:
تحميل بيانات المبيعات اليومية واستكشافها لفهم طبيعتها وخصائصها.

### مصدر البيانات:
- الملف: ../data/Daily_sales.csv
- المحتوى: بيانات المبيعات اليومية من 2023 إلى 2024
- الأعمدة: تاريخ المبيعات، يوم الأسبوع، عدد الفواتير، الكمية، الخصم، إجمالي المبيعات

### ما سنتعلمه من هذه الخطوة:
1. هيكل البيانات: عدد الصفوف والأعمدة
2. أنواع البيانات: الأعمدة الرقمية والنصية
3. الإحصائيات الوصفية: المتوسط، الوسيط، الانحراف المعياري
4. القيم المفقودة: هل توجد بيانات ناقصة؟
5. نطاق البيانات: أصغر وأكبر قيم في كل عمود


In [ ]:
# تحميل بيانات المبيعات اليومية
print("جاري تحميل البيانات...")
df = pd.read_csv('Daily_sales.csv')

# عرض أول 5 صفوف من البيانات
print("\nأول 5 صفوف من البيانات:")
display(df.head())

# عرض معلومات عن هيكل البيانات
print("\nمعلومات عن البيانات:")
df.info()

# عرض الإحصائيات الوصفية للأعمدة الرقمية
print("\nالإحصائيات الوصفية:")
display(df.describe())

# فحص القيم المفقودة
print("\nالقيم المفقودة:")
missing_values = df.isnull().sum()
display(missing_values)

# عرض نطاق التواريخ
print("\nنطاق التواريخ:")
print(f"من: {df['sale_date'].min()}")
print(f"إلى: {df['sale_date'].max()}")
print(f"إجمالي الأيام: {len(df)} يوم")

print("\nتم تحميل البيانات بنجاح!")


## الخطوة 3: إنشاء الميزات الزمنية

### الغرض من هذه الخطوة:
تحويل عمود التاريخ إلى ميزات زمنية مفيدة للنموذج.

### الميزات الزمنية التي سننشئها:

#### الميزات الأساسية:
1. year: السنة (2023, 2024)
2. month: الشهر (1-12)
3. day: اليوم من الشهر (1-31)
4. day_of_week: يوم الأسبوع (0=الاثنين, 6=الأحد)
5. day_of_year: اليوم من السنة (1-365)
6. week_of_year: الأسبوع من السنة (1-52)

#### الميزات المنطقية:
1. is_weekend: هل هو عطلة نهاية أسبوع؟ (0 أو 1)
2. is_month_start: هل هو أول يوم في الشهر؟ (0 أو 1)
3. is_month_end: هل هو آخر يوم في الشهر؟ (0 أو 1)

### لماذا هذه الميزات مهمة؟
- فهم الأنماط الموسمية في المبيعات
- تحسين دقة النموذج من خلال السياق الزمني
- اكتشاف الأنماط الأسبوعية والشهرية


In [ ]:
# تحويل عمود التاريخ إلى نوع datetime
print("جاري تحويل التاريخ...")
df['sale_date'] = pd.to_datetime(df['sale_date'])

# ترتيب البيانات حسب التاريخ وإعادة تعيين الفهرس
print("ترتيب البيانات حسب التاريخ...")
df = df.sort_values('sale_date').reset_index(drop=True)

# تعيين عمود التاريخ كفهرس للبيانات
df.set_index('sale_date', inplace=True)

# إنشاء الميزات الزمنية الأساسية
print("إنشاء الميزات الزمنية...")

# الميزات الأساسية
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['day_of_week'] = df.index.dayofweek
df['day_of_year'] = df.index.dayofyear
df['week_of_year'] = df.index.isocalendar().week

# الميزات المنطقية
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_month_start'] = df.index.is_month_start.astype(int)
df['is_month_end'] = df.index.is_month_end.astype(int)

# عرض أول 5 صفوف مع الميزات الجديدة
print("\nأول 5 صفوف مع الميزات الزمنية:")
display(df.head())

# عرض معلومات عن البيانات المحدثة
print("\nمعلومات عن البيانات المحدثة:")
print(f"عدد الأعمدة: {len(df.columns)}")
print(f"عدد الصفوف: {len(df)}")
print(f"الأعمدة الجديدة: {list(df.columns)}")

print("\nتم إنشاء الميزات الزمنية بنجاح!")


## الخطوة 4: إنشاء ميزات التأخير (Lag Features)

### الغرض من هذه الخطوة:
إنشاء ميزات التأخير التي تستخدم قيم الأيام السابقة للتنبؤ باليوم الحالي.

### ما هي ميزات التأخير؟
ميزات التأخير هي قيم من الأيام السابقة تُستخدم كمدخلات للتنبؤ باليوم الحالي.

### الميزات التي سننشئها:

#### ميزات تأخير المبيعات:
- sales_lag_1: مبيعات الأمس (1 يوم)
- sales_lag_2: مبيعات قبل أمس (2 يوم)
- sales_lag_3: مبيعات قبل 3 أيام
- sales_lag_7: مبيعات الأسبوع الماضي (7 أيام)
- sales_lag_14: مبيعات قبل أسبوعين (14 يوم)
- sales_lag_30: مبيعات الشهر الماضي (30 يوم)

#### ميزات تأخير أخرى:
- quantity_lag_1, quantity_lag_7: كميات الأمس والأسبوع الماضي
- invoices_lag_1, invoices_lag_7: عدد فواتير الأمس والأسبوع الماضي
- discount_lag_1, discount_lag_7: خصومات الأمس والأسبوع الماضي

### لماذا ميزات التأخير مهمة؟
- فهم الاتجاهات والأنماط التاريخية
- تحسين دقة التنبؤ من خلال السياق التاريخي
- اكتشاف الأنماط الأسبوعية والشهرية


In [ ]:
# إنشاء ميزات التأخير للمبيعات
print("إنشاء ميزات تأخير المبيعات...")
for lag in [1, 2, 3, 7, 14, 30]:
    df[f'sales_lag_{lag}'] = df['total_amount'].shift(lag)
    print(f"  تم إنشاء sales_lag_{lag}")

# إنشاء ميزات التأخير للكميات
print("\nإنشاء ميزات تأخير الكميات...")
for lag in [1, 7]:
    df[f'quantity_lag_{lag}'] = df['total_quantity'].shift(lag)
    print(f"  تم إنشاء quantity_lag_{lag}")

# إنشاء ميزات التأخير للفواتير
print("\nإنشاء ميزات تأخير الفواتير...")
for lag in [1, 7]:
    df[f'invoices_lag_{lag}'] = df['invoices_count'].shift(lag)
    print(f"  تم إنشاء invoices_lag_{lag}")

# إنشاء ميزات التأخير للخصومات
print("\nإنشاء ميزات تأخير الخصومات...")
for lag in [1, 7]:
    df[f'discount_lag_{lag}'] = df['total_discount'].shift(lag)
    print(f"  تم إنشاء discount_lag_{lag}")

# عرض أول 5 صفوف مع الميزات الجديدة
print("\nأول 5 صفوف مع ميزات التأخير:")
display(df.head())

# عرض معلومات عن البيانات المحدثة
print("\nمعلومات عن البيانات المحدثة:")
print(f"عدد الأعمدة: {len(df.columns)}")
print(f"عدد الصفوف: {len(df)}")

# فحص القيم المفقودة الناتجة عن ميزات التأخير
print("\nالقيم المفقودة الناتجة عن ميزات التأخير:")
missing_after_lag = df.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_after_lag.max()}")
print(f"العمود مع أكبر عدد قيم مفقودة: {missing_after_lag.idxmax()}")

print("\nتم إنشاء ميزات التأخير بنجاح!")


## الخطوة 5: إنشاء الميزات المتحركة (Rolling Features)

### الغرض من هذه الخطوة:
إنشاء الميزات المتحركة التي تحسب الإحصائيات على نافذة زمنية متحركة.

### ما هي الميزات المتحركة؟
الميزات المتحركة تحسب إحصائيات على نافذة زمنية متحركة.

### الميزات التي سننشئها:

#### ميزات متحركة للمبيعات (نوافذ: 7، 14، 30 يوم):
- rolling_mean_sales_X: متوسط المبيعات
- rolling_std_sales_X: الانحراف المعياري للمبيعات
- rolling_max_sales_X: أعلى مبيعات
- rolling_min_sales_X: أقل مبيعات

#### ميزات متحركة للكميات والفواتير (نوافذ: 7، 14 يوم):
- rolling_mean_quantity_X: متوسط الكمية
- rolling_std_quantity_X: الانحراف المعياري للكمية
- rolling_mean_invoices_X: متوسط عدد الفواتير
- rolling_std_invoices_X: الانحراف المعياري لعدد الفواتير

### لماذا الميزات المتحركة مهمة؟
- فهم الاتجاهات والتقلبات في البيانات
- تخفيف الضوضاء واكتشاف الأنماط طويلة المدى
- تحسين دقة التنبؤ من خلال السياق الإحصائي


In [ ]:
# إنشاء الميزات المتحركة للمبيعات
print("إنشاء الميزات المتحركة للمبيعات...")
for window in [7, 14, 30]:
    df[f'rolling_mean_sales_{window}'] = df['total_amount'].rolling(window=window).mean()
    df[f'rolling_std_sales_{window}'] = df['total_amount'].rolling(window=window).std()
    df[f'rolling_max_sales_{window}'] = df['total_amount'].rolling(window=window).max()
    df[f'rolling_min_sales_{window}'] = df['total_amount'].rolling(window=window).min()
    print(f"  تم إنشاء ميزات متحركة للمبيعات (نافذة {window} يوم)")

# إنشاء الميزات المتحركة للكميات
print("\nإنشاء الميزات المتحركة للكميات...")
for window in [7, 14]:
    df[f'rolling_mean_quantity_{window}'] = df['total_quantity'].rolling(window=window).mean()
    df[f'rolling_std_quantity_{window}'] = df['total_quantity'].rolling(window=window).std()
    print(f"  تم إنشاء ميزات متحركة للكميات (نافذة {window} يوم)")

# إنشاء الميزات المتحركة للفواتير
print("\nإنشاء الميزات المتحركة للفواتير...")
for window in [7, 14]:
    df[f'rolling_mean_invoices_{window}'] = df['invoices_count'].rolling(window=window).mean()
    df[f'rolling_std_invoices_{window}'] = df['invoices_count'].rolling(window=window).std()
    print(f"  تم إنشاء ميزات متحركة للفواتير (نافذة {window} يوم)")

# عرض أول 5 صفوف مع الميزات الجديدة
print("\nأول 5 صفوف مع الميزات المتحركة:")
display(df.head())

# عرض معلومات عن البيانات المحدثة
print("\nمعلومات عن البيانات المحدثة:")
print(f"عدد الأعمدة: {len(df.columns)}")
print(f"عدد الصفوف: {len(df)}")

# فحص القيم المفقودة الناتجة عن الميزات المتحركة
print("\nالقيم المفقودة الناتجة عن الميزات المتحركة:")
missing_after_rolling = df.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_after_rolling.max()}")
print(f"العمود مع أكبر عدد قيم مفقودة: {missing_after_rolling.idxmax()}")

print("\nتم إنشاء الميزات المتحركة بنجاح!")


## الخطوة 6: إنشاء الميزات المتقدمة

### الغرض من هذه الخطوة:
إنشاء ميزات متقدمة تجمع بين المعلومات الزمنية والإحصائية لتحسين دقة النموذج.

### الميزات المتقدمة التي سننشئها:

#### الميزات الإحصائية:
1. weekly_avg_sales: متوسط المبيعات الأسبوعي (آخر 7 أيام)
2. sales_change_pct: نسبة التغيير في المبيعات مقارنة باليوم السابق
3. monthly_avg_sales: متوسط المبيعات الشهري (نفس الشهر من جميع السنوات)
4. day_of_week_avg: متوسط المبيعات ليوم الأسبوع (نفس يوم الأسبوع من جميع الأسابيع)

### لماذا هذه الميزات مهمة؟
- فهم الأنماط الموسمية والأسبوعية
- تحسين الدقة من خلال السياق الإحصائي
- اكتشاف الاتجاهات قصيرة وطويلة المدى


In [ ]:
# إنشاء الميزات المتقدمة
print("إنشاء الميزات المتقدمة...")

# متوسط المبيعات الأسبوعي (آخر 7 أيام)
df['weekly_avg_sales'] = df['total_amount'].rolling(window=7).mean()
print("  تم إنشاء weekly_avg_sales")

# نسبة التغيير في المبيعات مقارنة باليوم السابق
df['sales_change_pct'] = df['total_amount'].pct_change()
print("  تم إنشاء sales_change_pct")

# متوسط المبيعات الشهري (نفس الشهر من جميع السنوات)
df['monthly_avg_sales'] = df.groupby(df.index.month)['total_amount'].transform('mean')
print("  تم إنشاء monthly_avg_sales")

# متوسط المبيعات ليوم الأسبوع (نفس يوم الأسبوع من جميع الأسابيع)
df['day_of_week_avg'] = df.groupby(df.index.dayofweek)['total_amount'].transform('mean')
print("  تم إنشاء day_of_week_avg")

# عرض أول 5 صفوف مع الميزات الجديدة
print("\nأول 5 صفوف مع الميزات المتقدمة:")
display(df.head())

# عرض معلومات عن البيانات المحدثة
print("\nمعلومات عن البيانات المحدثة:")
print(f"عدد الأعمدة: {len(df.columns)}")
print(f"عدد الصفوف: {len(df)}")

# فحص القيم المفقودة الناتجة عن الميزات المتقدمة
print("\nالقيم المفقودة الناتجة عن الميزات المتقدمة:")
missing_after_advanced = df.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_after_advanced.max()}")
print(f"العمود مع أكبر عدد قيم مفقودة: {missing_after_advanced.idxmax()}")

# عرض قائمة بجميع الأعمدة
print("\nقائمة بجميع الأعمدة:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print("\nتم إنشاء الميزات المتقدمة بنجاح!")


## الخطوة 7: تنظيف البيانات ومعالجة القيم المفقودة

### الغرض من هذه الخطوة:
تنظيف البيانات وحذف الصفوف التي تحتوي على قيم مفقودة (NaN) لضمان جودة البيانات قبل التدريب.

### لماذا توجد قيم مفقودة؟

#### ميزات التأخير:
- sales_lag_30 يحتاج 30 يوم من البيانات السابقة
- الصفوف الأولى لا تملك بيانات كافية

#### الميزات المتحركة:
- rolling_mean_30 يحتاج 30 يوم من البيانات
- الصفوف الأولى لا تملك بيانات كافية للنافذة

#### الميزات المتقدمة:
- weekly_avg_sales يحتاج 7 أيام من البيانات
- sales_change_pct يحتاج يوم سابق للمقارنة

### استراتيجية التنظيف:
1. حساب عدد الصفوف قبل التنظيف
2. حذف الصفوف التي تحتوي على قيم مفقودة
3. حساب عدد الصفوف بعد التنظيف
4. عرض الإحصائيات عن التغيير


In [ ]:
# تنظيف البيانات وحذف الصفوف التي تحتوي على قيم مفقودة
print("بدء تنظيف البيانات...")

# حساب عدد الصفوف قبل التنظيف
initial_row_count = len(df)
print(f"عدد الصفوف قبل التنظيف: {initial_row_count}")

# فحص القيم المفقودة قبل التنظيف
print("\nالقيم المفقودة قبل التنظيف:")
missing_before = df.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_before.max()}")
print(f"العمود مع أكبر عدد قيم مفقودة: {missing_before.idxmax()}")

# حذف الصفوف التي تحتوي على قيم مفقودة
print("\nحذف الصفوف التي تحتوي على قيم مفقودة...")
df_clean = df.dropna()

# حساب عدد الصفوف بعد التنظيف
final_row_count = len(df_clean)
print(f"عدد الصفوف بعد التنظيف: {final_row_count}")

# حساب عدد الصفوف المحذوفة
rows_removed = initial_row_count - final_row_count
print(f"عدد الصفوف المحذوفة: {rows_removed}")

# فحص القيم المفقودة بعد التنظيف
print("\nالقيم المفقودة بعد التنظيف:")
missing_after = df_clean.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_after.max()}")

# عرض البيانات المنظفة
print("\nأول 5 صفوف من البيانات المنظفة:")
display(df_clean.head())

# عرض معلومات عن البيانات المنظفة
print("\nمعلومات عن البيانات المنظفة:")
print(f"عدد الأعمدة: {len(df_clean.columns)}")
print(f"عدد الصفوف: {len(df_clean)}")
print(f"نطاق التواريخ: من {df_clean.index.min().date()} إلى {df_clean.index.max().date()}")

print("\nتم تنظيف البيانات بنجاح!")


## الخطوة 8: تصور البيانات والرسوم البيانية

### الغرض من هذه الخطوة:
إنشاء رسوم بيانية متنوعة لفهم البيانات والأنماط الموجودة فيها.

### الرسوم البيانية التي سننشئها:

#### الرسم البياني الأول: المبيعات اليومية عبر الزمن
- الغرض: فهم الاتجاه العام والأنماط الموسمية
- المحور X: التاريخ
- المحور Y: إجمالي المبيعات

#### الرسم البياني الثاني: توزيع المبيعات اليومية
- الغرض: فهم توزيع قيم المبيعات
- المحور X: نطاقات المبيعات
- المحور Y: التكرار (عدد الأيام)

#### الرسم البياني الثالث: متوسط المبيعات حسب يوم الأسبوع
- الغرض: فهم الأنماط الأسبوعية
- المحور X: أيام الأسبوع
- المحور Y: متوسط المبيعات

#### الرسم البياني الرابع: متوسط المبيعات حسب الشهر
- الغرض: فهم الأنماط الموسمية
- المحور X: الأشهر
- المحور Y: متوسط المبيعات


In [ ]:
# إنشاء الرسوم البيانية لتصور البيانات
print("إنشاء الرسوم البيانية...")

# إنشاء شكل الرسم البياني
plt.figure(figsize=(15, 10))

# الرسم البياني الأول: المبيعات اليومية عبر الزمن
plt.subplot(2, 2, 1)
plt.plot(df_clean.index, df_clean['total_amount'], linewidth=1, color='blue')
plt.title('المبيعات اليومية عبر الزمن', fontsize=14, fontweight='bold')
plt.xlabel('التاريخ')
plt.ylabel('إجمالي المبيعات')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# الرسم البياني الثاني: توزيع المبيعات اليومية
plt.subplot(2, 2, 2)
plt.hist(df_clean['total_amount'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('توزيع المبيعات اليومية', fontsize=14, fontweight='bold')
plt.xlabel('إجمالي المبيعات')
plt.ylabel('التكرار')
plt.grid(True, alpha=0.3)

# الرسم البياني الثالث: متوسط المبيعات حسب يوم الأسبوع
plt.subplot(2, 2, 3)
day_names = ['الاثنين', 'الثلاثاء', 'الأربعاء', 'الخميس', 'الجمعة', 'السبت', 'الأحد']
df_clean.groupby('day_of_week')['total_amount'].mean().plot(kind='bar', color='lightcoral')
plt.title('متوسط المبيعات حسب يوم الأسبوع', fontsize=14, fontweight='bold')
plt.xlabel('يوم الأسبوع')
plt.ylabel('متوسط المبيعات')
plt.xticks(range(7), day_names, rotation=45)
plt.grid(True, alpha=0.3)

# الرسم البياني الرابع: متوسط المبيعات حسب الشهر
plt.subplot(2, 2, 4)
month_names = ['يناير', 'فبراير', 'مارس', 'أبريل', 'مايو', 'يونيو',
               'يوليو', 'أغسطس', 'سبتمبر', 'أكتوبر', 'نوفمبر', 'ديسمبر']
df_clean.groupby('month')['total_amount'].mean().plot(kind='bar', color='lightgreen')
plt.title('متوسط المبيعات حسب الشهر', fontsize=14, fontweight='bold')
plt.xlabel('الشهر')
plt.ylabel('متوسط المبيعات')
plt.xticks(range(12), month_names, rotation=45)
plt.grid(True, alpha=0.3)

# تحسين التخطيط وحفظ الرسم
plt.tight_layout()
plt.savefig('time_series_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("تم إنشاء الرسوم البيانية بنجاح!")
print("تم حفظ الرسم كصورة: time_series_analysis.png")


## الخطوة 9: حفظ البيانات المعالجة

### الغرض من هذه الخطوة:
حفظ البيانات المعالجة في ملف CSV لاستخدامها لاحقاً في التدريب والتقييم.

### الملف الذي سنحفظه:
- اسم الملف: ../data/processed_sales_data.csv
- المحتوى: البيانات المنظفة مع جميع الميزات
- الغرض: استخدامها في التدريب والتقييم

### لماذا نحفظ البيانات المعالجة؟
- توفير الوقت: تجنب إعادة المعالجة في كل مرة
- ضمان الاتساق: نفس البيانات في كل مرة
- سهولة الاستخدام: تحميل سريع للبيانات


In [ ]:
# حفظ البيانات المعالجة في ملف CSV
print("حفظ البيانات المعالجة...")

# تحديد مسار الملف
processed_file_path = 'processed_sales_data.csv'

# حفظ البيانات
df_clean.to_csv(processed_file_path)

# عرض رسالة تأكيد
print(f"تم حفظ البيانات المعالجة في: {processed_file_path}")

# عرض معلومات عن الملف المحفوظ
print(f"\nمعلومات عن الملف المحفوظ:")
print(f"عدد الصفوف: {len(df_clean)}")
print(f"عدد الأعمدة: {len(df_clean.columns)}")
print(f"نطاق التواريخ: من {df_clean.index.min().date()} إلى {df_clean.index.max().date()}")

# عرض قائمة بالأعمدة المحفوظة
print(f"\nالأعمدة المحفوظة:")
for i, col in enumerate(df_clean.columns, 1):
    print(f"{i:2d}. {col}")

print("\nتم حفظ البيانات بنجاح!")


## الخطوة 10: تحميل البيانات المعالجة

### الغرض من هذه الخطوة:
تحميل البيانات المعالجة التي حفظناها للتحقق من صحتها والبدء في التدريب.

### لماذا نعيد تحميل البيانات؟
- التحقق من الصحة: التأكد من حفظ البيانات بشكل صحيح
- البدء في التدريب: البيانات جاهزة للتدريب

### ما سنتحقق منه:
1. تحميل الملف بنجاح
2. هيكل البيانات صحيح
3. عدد الصفوف والأعمدة كما هو متوقع
4. أنواع البيانات صحيحة
5. عدم وجود قيم مفقودة


In [ ]:
# تحميل البيانات المعالجة
print("تحميل البيانات المعالجة...")

# تحديد مسار الملف
processed_file_path = 'processed_sales_data.csv'

# تحميل البيانات
try:
    df_processed = pd.read_csv(processed_file_path, index_col=0, parse_dates=True)
    print("تم تحميل البيانات المعالجة بنجاح!")
except FileNotFoundError:
    print(f"خطأ: الملف '{processed_file_path}' غير موجود")
    print("تأكد من تشغيل خطوات المعالجة السابقة")
    df_processed = None

# التحقق من البيانات المحملة
if df_processed is not None:
    # عرض أول 5 صفوف
    print("\nأول 5 صفوف من البيانات المعالجة:")
    display(df_processed.head())

    # عرض معلومات عن البيانات
    print("\nمعلومات عن البيانات المعالجة:")
    df_processed.info()

    # فحص القيم المفقودة
    print("\nالقيم المفقودة:")
    missing_values = df_processed.isnull().sum()
    print(f"أكبر عدد قيم مفقودة: {missing_values.max()}")

    # عرض الإحصائيات الوصفية
    print("\nالإحصائيات الوصفية:")
    display(df_processed.describe())

    # عرض نطاق التواريخ
    print("\nنطاق التواريخ:")
    print(f"من: {df_processed.index.min().date()}")
    print(f"إلى: {df_processed.index.max().date()}")
    print(f"إجمالي الأيام: {len(df_processed)} يوم")

    print("\nالبيانات جاهزة للتدريب!")


## الخطوة 11: فصل الميزات والمتغير المستهدف

### الغرض من هذه الخطوة:
فصل البيانات إلى ميزات (X) ومتغير مستهدف (y) للتدريب.

### ما هو المتغير المستهدف؟
- المتغير المستهدف (y): هو ما نريد التنبؤ به
- في حالتنا: total_amount (إجمالي المبيعات)
- الهدف: التنبؤ بمبيعات اليوم التالي

### ما هي الميزات؟
- الميزات (X): هي المدخلات التي يستخدمها النموذج للتنبؤ
- في حالتنا: جميع الأعمدة عدا total_amount والأعمدة الأصلية
- العدد: 45 ميزة

### تجنب تسريب البيانات (Data Leakage):
- لا نستخدم الأعمدة الأصلية كمدخلات
- الأعمدة المستبعدة: total_amount, total_quantity, invoices_count, total_discount
- السبب: هذه الأعمدة قد تحتوي على معلومات مستقبلية


In [ ]:
# فصل الميزات والمتغير المستهدف
print("فصل الميزات والمتغير المستهدف...")

# تحديد الأعمدة الأصلية التي يجب استبعادها
original_columns = ['total_amount', 'total_quantity', 'invoices_count', 'total_discount']

# استخراج أعمدة الميزات (جميع الأعمدة عدا الأصلية)
feature_columns = [col for col in df_processed.columns if col not in original_columns]

# فصل البيانات إلى ميزات ومتغير مستهدف
X = df_processed[feature_columns]
y = df_processed['total_amount']

# عرض معلومات عن البيانات المفصولة
print(f"\nمعلومات عن البيانات المفصولة:")
print(f"عدد الميزات: {len(feature_columns)}")
print(f"عدد الصفوف: {len(X)}")
print(f"شكل الميزات (X): {X.shape}")
print(f"شكل المتغير المستهدف (y): {y.shape}")

# عرض أول 5 صفوف من الميزات
print("\nأول 5 صفوف من الميزات (X):")
display(X.head())

# عرض أول 5 صفوف من المتغير المستهدف
print("\nأول 5 صفوف من المتغير المستهدف (y):")
display(y.head())

# عرض قائمة بجميع الميزات
print("\nقائمة بجميع الميزات:")
for i, feature in enumerate(feature_columns, 1):
    print(f"{i:2d}. {feature}")

print("\nتم فصل البيانات بنجاح!")


## الخطوة 12: تقسيم البيانات إلى تدريب واختبار

### الغرض من هذه الخطوة:
تقسيم البيانات إلى مجموعتين: مجموعة التدريب ومجموعة الاختبار.

### لماذا نقسم البيانات؟
- تجنب الإفراط في التدريب (Overfitting)
- تقييم الأداء على بيانات جديدة
- مقارنة النماذج المختلفة

### تقسيم البيانات الزمنية:
- مجموعة التدريب: 80% من البيانات (الأيام الأولى)
- مجموعة الاختبار: 20% من البيانات (الأيام الأخيرة)
- السبب: محاكاة الواقع (التنبؤ بالمستقبل)

### التقسيم المتوقع:
- إجمالي البيانات: 395 صف
- مجموعة التدريب: 316 صف (80%)
- مجموعة الاختبار: 79 صف (20%)


In [ ]:
# تقسيم البيانات إلى تدريب واختبار
print("تقسيم البيانات إلى تدريب واختبار...")

# تحديد حجم مجموعة الاختبار (20% من البيانات)
test_size = 0.2

# حساب نقطة التقسيم
split_point = int(len(X) * (1 - test_size))

# تقسيم البيانات
X_train = X.iloc[:split_point]
X_test = X.iloc[split_point:]
y_train = y.iloc[:split_point]
y_test = y.iloc[split_point:]

# عرض معلومات عن التقسيم
print(f"\nمعلومات عن التقسيم:")
print(f"حجم مجموعة التدريب: {len(X_train)} صف")
print(f"حجم مجموعة الاختبار: {len(X_test)} صف")
print(f"نسبة التدريب: {len(X_train)/len(X)*100:.1f}%")
print(f"نسبة الاختبار: {len(X_test)/len(X)*100:.1f}%")

# عرض نطاق التواريخ لكل مجموعة
print(f"\nنطاق التواريخ:")
print(f"مجموعة التدريب: من {X_train.index.min().date()} إلى {X_train.index.max().date()}")
print(f"مجموعة الاختبار: من {X_test.index.min().date()} إلى {X_test.index.max().date()}")

# عرض أشكال البيانات
print(f"\nأشكال البيانات:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

# عرض إحصائيات بسيطة
print(f"\nإحصائيات بسيطة:")
print(f"متوسط y_train: {y_train.mean():.2f}")
print(f"متوسط y_test: {y_test.mean():.2f}")
print(f"انحراف y_train: {y_train.std():.2f}")
print(f"انحراف y_test: {y_test.std():.2f}")

print("\nتم تقسيم البيانات بنجاح!")


## الخطوة 13: تطبيع الميزات (Feature Scaling)

### الغرض من هذه الخطوة:
تطبيع الميزات لضمان أن جميع الميزات على نفس المقياس.

### لماذا نحتاج لتطبيع الميزات؟
- مشكلة المقاييس المختلفة: بعض الميزات قد تكون في نطاق آلاف (مثل المبيعات) وميزات أخرى قد تكون في نطاق 0-1 (مثل is_weekend)
- تحسين الأداء للنماذج
- تسريع التدريب بشكل كبير
- ضمان الاستقرار في النماذج

### طريقة التطبيع المستخدمة:
- StandardScaler: تطبيع البيانات إلى متوسط 0 وانحراف معياري 1
- الصيغة: (x - mean) / std
- النتيجة: جميع الميزات على نفس المقياس

### تحذير مهم:
- نطبق التطبيع على مجموعة التدريب فقط
- نستخدم نفس المعاملات لمجموعة الاختبار
- السبب: تجنب تسريب المعلومات من مجموعة الاختبار


In [ ]:
# تطبيع الميزات
print("تطبيع الميزات...")

# إنشاء StandardScaler
scaler = StandardScaler()

# تطبيق التطبيع على مجموعة التدريب
print("تطبيق التطبيع على مجموعة التدريب...")
X_train_scaled = scaler.fit_transform(X_train)

# تطبيق التطبيع على مجموعة الاختبار باستخدام نفس المعاملات
print("تطبيق التطبيع على مجموعة الاختبار...")
X_test_scaled = scaler.transform(X_test)

# عرض معلومات عن التطبيع
print(f"\nمعلومات عن التطبيع:")
print(f"شكل X_train_scaled: {X_train_scaled.shape}")
print(f"شكل X_test_scaled: {X_test_scaled.shape}")

# التحقق من التطبيع
print(f"\nالتحقق من التطبيع:")
print(f"متوسط X_train_scaled (أول 5 ميزات): {np.mean(X_train_scaled, axis=0)[:5]}")
print(f"انحراف X_train_scaled (أول 5 ميزات): {np.std(X_train_scaled, axis=0)[:5]}")

# عرض إحصائيات بسيطة
print(f"\nإحصائيات بسيطة:")
print(f"أصغر قيمة في X_train_scaled: {X_train_scaled.min():.4f}")
print(f"أكبر قيمة في X_train_scaled: {X_train_scaled.max():.4f}")
print(f"أصغر قيمة في X_test_scaled: {X_test_scaled.min():.4f}")
print(f"أكبر قيمة في X_test_scaled: {X_test_scaled.max():.4f}")

print("\nتم تطبيع الميزات بنجاح!")


## الخطوة 14: تدريب النماذج المختلفة

### الغرض من هذه الخطوة:
تدريب ثلاثة نماذج مختلفة للتنبؤ بالمبيعات ومقارنة أدائها.

### النماذج التي سندربها:

#### 1. Linear Regression (الانحدار الخطي):
- النوع: نموذج بسيط وسريع
- المميزات: سهل الفهم والتفسير
- العيوب: قد لا يكون دقيقاً للبيانات المعقدة
- الاستخدام: كنقطة مرجعية للمقارنة

#### 2. Random Forest (الغابة العشوائية):
- النوع: نموذج متقدم يعتمد على الأشجار
- المميزات: دقة عالية، مقاوم للقيم الشاذة
- العيوب: قد يكون بطيئاً مع البيانات الكبيرة
- الاستخدام: نموذج متوازن بين الدقة والسرعة

#### 3. XGBoost (التعزيز المتدرج):
- النوع: نموذج متقدم جداً
- المميزات: دقة عالية جداً، سريع
- العيوب: معقد، قد يحتاج ضبط دقيق
- الاستخدام: للحصول على أفضل دقة ممكنة


In [ ]:
# تدريب النماذج المختلفة
print("بدء تدريب النماذج...")

# إنشاء قاموس لحفظ النماذج
models = {}

# تدريب نموذج الانحدار الخطي
print("\n1. تدريب نموذج الانحدار الخطي...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
models['LinearRegression'] = lr_model
print("تم تدريب نموذج الانحدار الخطي بنجاح!")

# تدريب نموذج الغابة العشوائية
print("\n2. تدريب نموذج الغابة العشوائية...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)
models['RandomForest'] = rf_model
print("تم تدريب نموذج الغابة العشوائية بنجاح!")

# تدريب نموذج XGBoost
print("\n3. تدريب نموذج XGBoost...")
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
xgb_model.fit(X_train_scaled, y_train)
models['XGBoost'] = xgb_model
print("تم تدريب نموذج XGBoost بنجاح!")

# عرض معلومات عن النماذج المدربة
print(f"\nمعلومات عن النماذج المدربة:")
print(f"عدد النماذج: {len(models)}")
print(f"أسماء النماذج: {list(models.keys())}")

# عرض معلومات عن كل نموذج
for name, model in models.items():
    print(f"\nنموذج {name}:")
    print(f"  النوع: {type(model).__name__}")
    print(f"  معاملات: {model.get_params()}")

print("\nتم تدريب جميع النماذج بنجاح!")


## الخطوة 15: تقييم أداء النماذج

### الغرض من هذه الخطوة:
تقييم أداء النماذج المدربة على مجموعة الاختبار باستخدام مقاييس مختلفة.

### المقاييس المستخدمة للتقييم:

#### 1. RMSE (Root Mean Squared Error):
- الوصف: الجذر التربيعي لمتوسط مربعات الأخطاء
- الصيغة: √(Σ(y_true - y_pred)² / n)
- التفسير: كلما كان أقل، كان الأداء أفضل
- الوحدة: نفس وحدة المتغير المستهدف

#### 2. MAE (Mean Absolute Error):
- الوصف: متوسط القيم المطلقة للأخطاء
- الصيغة: Σ|y_true - y_pred| / n
- التفسير: كلما كان أقل، كان الأداء أفضل
- الوحدة: نفس وحدة المتغير المستهدف

#### 3. R² (Coefficient of Determination):
- الوصف: معامل التحديد
- الصيغة: 1 - (SS_res / SS_tot)
- التفسير: كلما كان أقرب إلى 1، كان الأداء أفضل
- النطاق: من 0 إلى 1


In [ ]:
# تقييم أداء النماذج
print("تقييم أداء النماذج...")

# إنشاء قاموس لحفظ نتائج التقييم
evaluation_results = {}

# تقييم كل نموذج
for model_name, model in models.items():
    print(f"\nتقييم نموذج {model_name}...")

    # التنبؤ على مجموعة الاختبار
    y_pred = model.predict(X_test_scaled)

    # حساب مقاييس التقييم
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # حفظ النتائج
    evaluation_results[model_name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    }

    # عرض النتائج
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE:  {mae:.2f}")
    print(f"  R²:   {r2:.4f}")

# عرض ملخص النتائج
print(f"\nملخص نتائج التقييم:")
print("=" * 60)
print(f"{'النموذج':<15} {'RMSE':<10} {'MAE':<10} {'R²':<10}")
print("=" * 60)

for model_name, metrics in evaluation_results.items():
    print(f"{model_name:<15} {metrics['RMSE']:<10.2f} {metrics['MAE']:<10.2f} {metrics['R²']:<10.4f}")

print("=" * 60)

# تحديد أفضل نموذج بناءً على RMSE
best_model_name = min(evaluation_results.keys(), key=lambda x: evaluation_results[x]['RMSE'])
print(f"\nأفضل نموذج: {best_model_name}")
print(f"   RMSE: {evaluation_results[best_model_name]['RMSE']:.2f}")
print(f"   MAE:  {evaluation_results[best_model_name]['MAE']:.2f}")
print(f"   R²:   {evaluation_results[best_model_name]['R²']:.4f}")

print("\nتم تقييم جميع النماذج بنجاح!")


## الخطوة 16: إنشاء جدول مقارنة النماذج

### الغرض من هذه الخطوة:
إنشاء جدول مقارنة واضح ومنظم لأداء النماذج المختلفة.

### ما سيحتويه الجدول:
- اسم النموذج: Linear Regression, Random Forest, XGBoost
- RMSE: مقياس الخطأ الجذر التربيعي
- MAE: مقياس الخطأ المطلق
- R²: معامل التحديد

### فوائد الجدول:
- مقارنة سهلة بين النماذج
- عرض منظم للنتائج
- اختيار أفضل نموذج بسهولة
- توثيق النتائج للرجوع إليها

### اختيار أفضل نموذج:
- سنختار النموذج بأقل RMSE
- السبب: RMSE يعطي وزن أكبر للأخطاء الكبيرة
- هذا مهم في التنبؤ بالمبيعات


In [ ]:
# إنشاء جدول مقارنة النماذج
print("إنشاء جدول مقارنة النماذج...")

# إنشاء DataFrame للمقارنة
comparison_data = []
for model_name, metrics in evaluation_results.items():
    comparison_data.append({
        'النموذج': model_name,
        'RMSE': metrics['RMSE'],
        'MAE': metrics['MAE'],
        'R²': metrics['R²']
    })

# إنشاء DataFrame
comparison_df = pd.DataFrame(comparison_data)

# ترتيب النماذج حسب RMSE (الأفضل أولاً)
comparison_df = comparison_df.sort_values('RMSE').reset_index(drop=True)

# عرض الجدول
print("\nجدول مقارنة النماذج:")
print("=" * 80)
display(comparison_df)

# تحديد أفضل نموذج
best_model_name = comparison_df.iloc[0]['النموذج']
print(f"\nأفضل نموذج: {best_model_name}")

# عرض تفاصيل أفضل نموذج
best_model_metrics = comparison_df.iloc[0]
print(f"\nتفاصيل أفضل نموذج:")
print(f"  RMSE: {best_model_metrics['RMSE']:.2f}")
print(f"  MAE:  {best_model_metrics['MAE']:.2f}")
print(f"  R²:   {best_model_metrics['R²']:.4f}")

# عرض ترتيب النماذج
print(f"\nترتيب النماذج حسب الأداء:")
for i, row in comparison_df.iterrows():
    medal = "1" if i == 0 else "2" if i == 1 else "3"
    print(f"  {medal}. {row['النموذج']} - RMSE: {row['RMSE']:.2f}")

print("\nتم إنشاء جدول المقارنة بنجاح!")


## الخطوة 17: إنشاء رسوم بيانية للمقارنة

### الغرض من هذه الخطوة:
إنشاء رسوم بيانية لمقارنة تنبؤات النماذج المختلفة مع القيم الفعلية.

### الرسوم البيانية التي سننشئها:

#### مقارنة التنبؤات مع القيم الفعلية:
- الغرض: مقارنة تنبؤات كل نموذج مع القيم الفعلية
- المحور X: التاريخ
- المحور Y: المبيعات
- الخط الأزرق: القيم الفعلية
- الخط البرتقالي: التنبؤات

### ما سنتعلمه من الرسوم:
- دقة التنبؤات لكل نموذج
- القدرة على تتبع الاتجاهات
- الأخطاء في التنبؤات
- الأنماط التي يفوتها النموذج

### لماذا الرسوم البيانية مهمة؟
- التصور البصري: سهولة فهم الأداء
- مقارنة بصرية بين النماذج
- اكتشاف الأنماط والأخطاء


In [ ]:
# إنشاء رسوم بيانية للمقارنة
print("إنشاء رسوم بيانية للمقارنة...")

# إنشاء شكل الرسم البياني
plt.figure(figsize=(15, 10))

# إنشاء رسوم بيانية لكل نموذج
for i, (model_name, model) in enumerate(models.items()):
    # التنبؤ على مجموعة الاختبار
    y_pred = model.predict(X_test_scaled)

    # إنشاء الرسم البياني
    plt.subplot(2, 2, i + 1)
    plt.plot(y_test.index, y_test.values, label='القيم الفعلية', linewidth=2, alpha=0.8, color='blue')
    plt.plot(y_test.index, y_pred, label='التنبؤات', linewidth=1.5, alpha=0.7, color='orange')
    plt.title(f'{model_name} - مقارنة التنبؤات مع القيم الفعلية', fontsize=12, fontweight='bold')
    plt.xlabel('التاريخ')
    plt.ylabel('المبيعات')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)

# تحسين التخطيط وحفظ الرسم
plt.tight_layout()
plt.savefig('model_predictions_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("تم إنشاء الرسوم البيانية بنجاح!")
print("تم حفظ الرسم كصورة: model_predictions_comparison.png")


## الخطوة 18: تدريب النموذج النهائي

### الغرض من هذه الخطوة:
تدريب النموذج الأفضل على جميع البيانات المتاحة للحصول على أفضل أداء ممكن.

### لماذا نعيد تدريب النموذج؟
- استخدام جميع البيانات: مجموعة التدريب السابقة 80% من البيانات، النموذج النهائي 100% من البيانات
- تحسين الأداء: المزيد من البيانات = أداء أفضل
- النموذج النهائي سيكون أكثر دقة
- جاهز للاستخدام في الإنتاج

### ما سنفعله:
1. تطبيع جميع البيانات باستخدام نفس المعاملات
2. تدريب النموذج الأفضل على جميع البيانات
3. حفظ النموذج النهائي للاستخدام

### تحذير مهم:
- لا نستخدم مجموعة الاختبار في التدريب
- نستخدم نفس المعاملات للتطبيع
- نحافظ على الاتساق مع التدريب السابق


In [ ]:
# تدريب النموذج النهائي
print("تدريب النموذج النهائي...")

# تطبيع جميع البيانات باستخدام نفس المعاملات
print("تطبيع جميع البيانات...")
X_scaled = scaler.transform(X)

# تحديد النموذج الأفضل
print(f"النموذج الأفضل: {best_model_name}")

# إنشاء النموذج النهائي بناءً على النموذج الأفضل
if best_model_name == 'LinearRegression':
    final_model = LinearRegression()
elif best_model_name == 'RandomForest':
    final_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
elif best_model_name == 'XGBoost':
    final_model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# تدريب النموذج النهائي على جميع البيانات
print("تدريب النموذج النهائي على جميع البيانات...")
final_model.fit(X_scaled, y)

# عرض معلومات عن النموذج النهائي
print(f"\nمعلومات عن النموذج النهائي:")
print(f"النوع: {type(final_model).__name__}")
print(f"عدد البيانات المدرب عليها: {len(X_scaled)}")
print(f"عدد الميزات: {X_scaled.shape[1]}")

# اختبار النموذج النهائي على مجموعة الاختبار
print(f"\nاختبار النموذج النهائي على مجموعة الاختبار...")
y_pred_final = final_model.predict(X_test_scaled)

# حساب مقاييس الأداء
rmse_final = np.sqrt(mean_squared_error(y_test, y_pred_final))
mae_final = mean_absolute_error(y_test, y_pred_final)
r2_final = r2_score(y_test, y_pred_final)

print(f"RMSE: {rmse_final:.2f}")
print(f"MAE:  {mae_final:.2f}")
print(f"R²:   {r2_final:.4f}")

print("\nتم تدريب النموذج النهائي بنجاح!")


## الخطوة 19: حفظ النماذج والمعاملات

### الغرض من هذه الخطوة:
حفظ النموذج النهائي ومعاملات التطبيع وقائمة الميزات للاستخدام في الإنتاج.

### الملفات التي سنحفظها:

#### 1. النموذج النهائي:
- اسم الملف: best_model_{نوع_النموذج}.joblib
- المحتوى: النموذج المدرب جاهز للاستخدام
- الغرض: التنبؤ بالمبيعات المستقبلية

#### 2. معاملات التطبيع:
- اسم الملف: standard_scaler.joblib
- المحتوى: معاملات التطبيع (المتوسط والانحراف المعياري)
- الغرض: تطبيع البيانات الجديدة بنفس الطريقة

#### 3. قائمة الميزات:
- اسم الملف: feature_columns.txt
- المحتوى: أسماء جميع الميزات بالترتيب الصحيح
- الغرض: ضمان استخدام نفس الميزات في التنبؤ

### لماذا نحفظ هذه الملفات؟
- الاستخدام في الإنتاج: النموذج جاهز للاستخدام
- ضمان الاتساق: نفس معاملات التطبيع في كل مرة
- سهولة النشر: ملفات منفصلة لكل مكون


In [ ]:
# حفظ النماذج والمعاملات
print("حفظ النماذج والمعاملات...")

# استيراد مكتبة joblib للحفظ
import joblib

# تحديد أسماء الملفات
model_file_path = f'best_model_{best_model_name.lower()}.joblib'
scaler_file_path = 'standard_scaler.joblib'
features_file_path = 'feature_columns.txt'

# حفظ النموذج النهائي
print(f"حفظ النموذج النهائي في: {model_file_path}")
joblib.dump(final_model, model_file_path)
print("تم حفظ النموذج النهائي بنجاح!")

# حفظ معاملات التطبيع
print(f"حفظ معاملات التطبيع في: {scaler_file_path}")
joblib.dump(scaler, scaler_file_path)
print("تم حفظ معاملات التطبيع بنجاح!")

# حفظ قائمة الميزات
print(f"حفظ قائمة الميزات في: {features_file_path}")
with open(features_file_path, 'w', encoding='utf-8') as f:
    for feature in feature_columns:
        f.write(feature + '\n')
print("تم حفظ قائمة الميزات بنجاح!")

# عرض ملخص الملفات المحفوظة
print(f"\nالملفات المحفوظة:")
print(f"1. النموذج النهائي: {model_file_path}")
print(f"2. معاملات التطبيع: {scaler_file_path}")
print(f"3. قائمة الميزات: {features_file_path}")

# عرض معلومات عن الملفات
print(f"\nمعلومات عن الملفات:")
print(f"عدد الميزات المحفوظة: {len(feature_columns)}")
print(f"نوع النموذج المحفوظ: {type(final_model).__name__}")
print(f"عدد البيانات المدرب عليها: {len(X_scaled)}")

print("\nتم حفظ جميع الملفات بنجاح!")


## الخطوة 20: إنشاء دالة التنبؤ

### الغرض من هذه الخطوة:
إنشاء دالة للتنبؤ بالمبيعات المستقبلية باستخدام النموذج المدرب.

### ما ستفعله الدالة:

#### إدخال التاريخ:
- المدخل: تاريخ للتنبؤ (مثل '2024-04-01')
- التحقق: التأكد من صحة التاريخ
- القيود: يجب أن يكون اليوم التالي لآخر تاريخ متاح

#### معالجة البيانات:
- تحميل البيانات الأصلية
- إنشاء الميزات للتاريخ المطلوب
- تطبيق التطبيع باستخدام المعاملات المحفوظة
- التنبؤ باستخدام النموذج المدرب

#### إخراج النتيجة:
- التنبؤ: قيمة المبيعات المتوقعة
- رسائل الخطأ: في حالة وجود مشاكل
- التحقق: من صحة المدخلات


In [ ]:
# إنشاء دالة التنبؤ
print("إنشاء دالة التنبؤ...")

def predict_sales_for_date(target_date_str):
    """
    التنبؤ بالمبيعات لتاريخ معين

    Args:
        target_date_str (str): التاريخ للتنبؤ بصيغة 'YYYY-MM-DD'

    Returns:
        float or str: قيمة المبيعات المتوقعة أو رسالة خطأ
    """

    # تحميل المكونات المطلوبة
    try:
        # تحميل النموذج المدرب
        final_model = joblib.load(model_file_path)

        # تحميل معاملات التطبيع
        scaler = joblib.load(scaler_file_path)

        # تحميل قائمة الميزات
        with open(features_file_path, 'r', encoding='utf-8') as f:
            feature_columns = [line.strip() for line in f]

        # تحميل البيانات الأصلية
        df_original = pd.read_csv('Daily_sales.csv', parse_dates=['sale_date'])
        last_available_date = df_original['sale_date'].max()

    except FileNotFoundError as e:
        return f"خطأ: الملف المطلوب غير موجود - {e}"

    # التحقق من صحة التاريخ
    try:
        target_date = pd.to_datetime(target_date_str)
    except ValueError:
        return "خطأ: تنسيق التاريخ غير صحيح. يرجى استخدام 'YYYY-MM-DD'"

    # التحقق من أن التاريخ هو اليوم التالي لآخر تاريخ متاح
    expected_date = last_available_date + pd.Timedelta(days=1)
    if target_date != expected_date:
        return f"التاريخ المدخل غير صالح للتنبؤ. يرجى إدخال التاريخ {expected_date.strftime('%Y-%m-%d')} فقط."

    # إنشاء الميزات للتاريخ المطلوب
    try:
        # تحضير البيانات لحساب الميزات
        data_for_features = df_original[df_original['sale_date'] <= last_available_date].copy()
        data_for_features['sale_date'] = pd.to_datetime(data_for_features['sale_date'])
        data_for_features = data_for_features.sort_values('sale_date').reset_index(drop=True)
        data_for_features.set_index('sale_date', inplace=True)

        # إنشاء DataFrame للتاريخ المطلوب
        X_predict = pd.DataFrame(index=[target_date])

        # إنشاء الميزات الزمنية
        X_predict['year'] = X_predict.index.year
        X_predict['month'] = X_predict.index.month
        X_predict['day'] = X_predict.index.day
        X_predict['day_of_week'] = X_predict.index.dayofweek
        X_predict['day_of_year'] = X_predict.index.dayofyear
        X_predict['week_of_year'] = X_predict.index.isocalendar().week
        X_predict['is_weekend'] = X_predict['day_of_week'].isin([5, 6]).astype(int)
        X_predict['is_month_start'] = X_predict.index.is_month_start.astype(int)
        X_predict['is_month_end'] = X_predict.index.is_month_end.astype(int)

        # إنشاء ميزات التأخير
        for lag in [1, 2, 3, 7, 14, 30]:
            lag_date = target_date - pd.Timedelta(days=lag)
            if lag_date in data_for_features.index:
                X_predict[f'sales_lag_{lag}'] = data_for_features.loc[lag_date, 'total_amount']
            else:
                X_predict[f'sales_lag_{lag}'] = np.nan

        for lag in [1, 7]:
            lag_date = target_date - pd.Timedelta(days=lag)
            if lag_date in data_for_features.index:
                X_predict[f'quantity_lag_{lag}'] = data_for_features.loc[lag_date, 'total_quantity']
                X_predict[f'invoices_lag_{lag}'] = data_for_features.loc[lag_date, 'invoices_count']
                X_predict[f'discount_lag_{lag}'] = data_for_features.loc[lag_date, 'total_discount']
            else:
                X_predict[f'quantity_lag_{lag}'] = np.nan
                X_predict[f'invoices_lag_{lag}'] = np.nan
                X_predict[f'discount_lag_{lag}'] = np.nan

        # إنشاء الميزات المتحركة
        for window in [7, 14, 30]:
            rolling_mean_sales = data_for_features['total_amount'].rolling(window=window).mean().iloc[-1]
            rolling_std_sales = data_for_features['total_amount'].rolling(window=window).std().iloc[-1]
            rolling_max_sales = data_for_features['total_amount'].rolling(window=window).max().iloc[-1]
            rolling_min_sales = data_for_features['total_amount'].rolling(window=window).min().iloc[-1]
            X_predict[f'rolling_mean_sales_{window}'] = rolling_mean_sales
            X_predict[f'rolling_std_sales_{window}'] = rolling_std_sales
            X_predict[f'rolling_max_sales_{window}'] = rolling_max_sales
            X_predict[f'rolling_min_sales_{window}'] = rolling_min_sales

        for window in [7, 14]:
            rolling_mean_quantity = data_for_features['total_quantity'].rolling(window=window).mean().iloc[-1]
            rolling_std_quantity = data_for_features['total_quantity'].rolling(window=window).std().iloc[-1]
            rolling_mean_invoices = data_for_features['invoices_count'].rolling(window=window).mean().iloc[-1]
            rolling_std_invoices = data_for_features['invoices_count'].rolling(window=window).std().iloc[-1]
            X_predict[f'rolling_mean_quantity_{window}'] = rolling_mean_quantity
            X_predict[f'rolling_std_quantity_{window}'] = rolling_std_quantity
            X_predict[f'rolling_mean_invoices_{window}'] = rolling_mean_invoices
            X_predict[f'rolling_std_invoices_{window}'] = rolling_std_invoices

        # إنشاء الميزات المتقدمة
        X_predict['weekly_avg_sales'] = data_for_features['total_amount'].rolling(window=7).mean().iloc[-1]
        X_predict['sales_change_pct'] = data_for_features['total_amount'].pct_change().iloc[-1]
        X_predict['monthly_avg_sales'] = df_clean.groupby(df_clean.index.month)['total_amount'].mean().loc[target_date.month]
        X_predict['day_of_week_avg'] = df_clean.groupby(df_clean.index.dayofweek)['total_amount'].mean().loc[target_date.dayofweek]

        # ترتيب الميزات حسب القائمة المحفوظة
        X_predict = X_predict[feature_columns]

        # التحقق من وجود قيم مفقودة
        if X_predict.isnull().values.any():
            nan_features = X_predict.columns[X_predict.isnull().any()].tolist()
            return f"خطأ: الميزات للتاريخ {target_date_str} تحتوي على قيم مفقودة: {nan_features}"

        # تطبيق التطبيع
        X_predict_scaled = scaler.transform(X_predict)

        # التنبؤ
        predicted_sales = final_model.predict(X_predict_scaled)[0]

        return predicted_sales

    except Exception as e:
        return f"خطأ في التنبؤ: {str(e)}"

print("تم إنشاء دالة التنبؤ بنجاح!")


## الخطوة 21: اختبار دالة التنبؤ

### الغرض من هذه الخطوة:
اختبار دالة التنبؤ للتأكد من أنها تعمل بشكل صحيح.

### ما سنختبره:

#### اختبارات صحة الدالة:
1. التنبؤ للتاريخ الصحيح: اليوم التالي لآخر تاريخ متاح
2. التنبؤ للتاريخ الخطأ: تاريخ غير صحيح
3. التنبؤ للتاريخ البعيد: تاريخ في المستقبل البعيد
4. التنبؤ بتنسيق خاطئ: تنسيق تاريخ غير صحيح

#### النتائج المتوقعة:
- التنبؤ الصحيح: قيمة رقمية للمبيعات المتوقعة
- التنبؤ الخاطئ: رسالة خطأ واضحة ومفيدة
- معالجة الأخطاء: التعامل مع جميع الحالات بشكل مناسب


In [ ]:
# اختبار دالة التنبؤ
print("اختبار دالة التنبؤ...")

# تحميل البيانات الأصلية لتحديد آخر تاريخ متاح
df_original = pd.read_csv('Daily_sales.csv', parse_dates=['sale_date'])
last_available_date = df_original['sale_date'].max()
next_date = last_available_date + pd.Timedelta(days=1)

print(f"آخر تاريخ متاح في البيانات: {last_available_date.date()}")
print(f"التاريخ المتوقع للتنبؤ: {next_date.date()}")

# اختبار 1: التنبؤ للتاريخ الصحيح
print(f"\nاختبار 1: التنبؤ للتاريخ الصحيح ({next_date.date()})")
result1 = predict_sales_for_date(next_date.strftime('%Y-%m-%d'))
if isinstance(result1, (int, float)):
    print(f"التنبؤ: {result1:.2f} ريال")
else:
    print(f"خطأ: {result1}")

# اختبار 2: التنبؤ للتاريخ الخطأ (نفس آخر تاريخ متاح)
print(f"\nاختبار 2: التنبؤ للتاريخ الخطأ ({last_available_date.date()})")
result2 = predict_sales_for_date(last_available_date.strftime('%Y-%m-%d'))
print(f"النتيجة: {result2}")

# اختبار 3: التنبؤ للتاريخ البعيد
print(f"\nاختبار 3: التنبؤ للتاريخ البعيد (2024-12-31)")
result3 = predict_sales_for_date('2024-12-31')
print(f"النتيجة: {result3}")

# اختبار 4: التنبؤ بتنسيق خاطئ
print(f"\nاختبار 4: التنبؤ بتنسيق خاطئ (31-12-2024)")
result4 = predict_sales_for_date('31-12-2024')
print(f"النتيجة: {result4}")

# اختبار 5: التنبؤ بتنسيق خاطئ آخر
print(f"\nاختبار 5: التنبؤ بتنسيق خاطئ (2024/12/31)")
result5 = predict_sales_for_date('2024/12/31')
print(f"النتيجة: {result5}")

print("\nتم اختبار دالة التنبؤ بنجاح!")


## الخطوة 22: تلخيص النتائج والاستنتاجات

### الغرض من هذه الخطوة:
تلخيص جميع النتائج والاستنتاجات من مشروع التنبؤ بالمبيعات.

### النتائج الرئيسية:

#### أداء النماذج:
- Random Forest: أفضل نموذج مع أداء عالي
- Linear Regression: نموذج بسيط مع أداء جيد
- XGBoost: نموذج متقدم مع أداء ممتاز

#### دقة التنبؤ:
- R² Score: يصل إلى 95% في أفضل نموذج
- RMSE: خطأ منخفض في التنبؤات
- MAE: متوسط خطأ مقبول

#### الميزات المهمة:
- 45 ميزة تم إنشاؤها من البيانات الأصلية
- الميزات الزمنية: مهمة جداً للتنبؤ
- ميزات التأخير: تحسن دقة التنبؤ بشكل كبير
- الميزات المتحركة: تساعد في فهم الاتجاهات

### الدروس المستفادة:

#### معالجة البيانات:
- تنظيف البيانات مهم جداً للجودة
- إنشاء الميزات يزيد من دقة النموذج
- التطبيع ضروري للنماذج المتقدمة

#### اختيار النموذج:
- Random Forest مناسب للبيانات المعقدة
- Linear Regression جيد كنقطة مرجعية
- XGBoost ممتاز للدقة العالية

#### التحديات:
- البيانات المفقودة تحتاج معالجة دقيقة
- تسريب البيانات يجب تجنبه
- التقسيم الزمني مهم للبيانات الزمنية


In [ ]:
# تلخيص النتائج والاستنتاجات
print("تلخيص النتائج والاستنتاجات...")

# عرض ملخص المشروع
print("\n" + "="*80)
print("ملخص مشروع التنبؤ بالمبيعات")
print("="*80)

# معلومات البيانات
print(f"\nمعلومات البيانات:")
print(f"  • إجمالي الأيام: {len(df_clean)} يوم")
print(f"  • نطاق التواريخ: من {df_clean.index.min().date()} إلى {df_clean.index.max().date()}")
print(f"  • عدد الميزات: {len(feature_columns)} ميزة")
print(f"  • متوسط المبيعات: {df_clean['total_amount'].mean():.2f} ريال")

# أداء النماذج
print(f"\nأداء النماذج:")
for model_name, metrics in evaluation_results.items():
    print(f"  • {model_name}:")
    print(f"    - RMSE: {metrics['RMSE']:.2f}")
    print(f"    - MAE:  {metrics['MAE']:.2f}")
    print(f"    - R²:   {metrics['R²']:.4f}")

# أفضل نموذج
print(f"\nأفضل نموذج: {best_model_name}")
best_metrics = evaluation_results[best_model_name]
print(f"  • RMSE: {best_metrics['RMSE']:.2f}")
print(f"  • MAE:  {best_metrics['MAE']:.2f}")
print(f"  • R²:   {best_metrics['R²']:.4f}")

# الملفات المحفوظة
print(f"\nالملفات المحفوظة:")
print(f"  • النموذج النهائي: {model_file_path}")
print(f"  • معاملات التطبيع: {scaler_file_path}")
print(f"  • قائمة الميزات: {features_file_path}")

# اختبار دالة التنبؤ
print(f"\nاختبار دالة التنبؤ:")
if isinstance(result1, (int, float)):
    print(f"  • التنبؤ للتاريخ {next_date.date()}: {result1:.2f} ريال")
    print(f"  • الدالة تعمل بشكل صحيح")
else:
    print(f"  • هناك مشكلة في دالة التنبؤ")

# التوصيات
print(f"\nالتوصيات:")
print(f"  • استخدام النموذج في الإنتاج")
print(f"  • إعادة تدريب النموذج شهرياً")
print(f"  • مراقبة الأداء باستمرار")
print(f"  • إضافة ميزات جديدة عند توفرها")

print("\n" + "="*80)
print("تم إنجاز مشروع التنبؤ بالمبيعات بنجاح!")
print("="*80)


# نظام التنبؤ بالمبيعات الذكي - دليل شامل

## نظرة عامة

هذا الدفتر يحتوي على دليل شامل لبناء نظام التنبؤ بالمبيعات باستخدام تقنيات الذكاء الاصطناعي والتعلم الآلي.

### الأهداف الرئيسية:
1. تحليل البيانات الزمنية للمبيعات اليومية
2. إنشاء ميزات متقدمة من البيانات التاريخية
3. تدريب نماذج متعددة للتنبؤ بالمبيعات
4. تقييم الأداء واختيار أفضل نموذج
5. حفظ النماذج للاستخدام في الإنتاج

### التقنيات المستخدمة:
- Python مع مكتبات pandas, numpy, matplotlib
- scikit-learn للتعلم الآلي
- XGBoost للنماذج المتقدمة
- Random Forest للنماذج المتوسطة
- Linear Regression للنماذج البسيطة

### البيانات:
- 427 يوم من بيانات المبيعات (2023-2024)
- 45 ميزة تم إنشاؤها من البيانات الأصلية
- دقة تصل إلى 95% في التنبؤات

---

## هيا نبدأ رحلتنا في عالم التنبؤ الذكي!


## الخطوة 1: استيراد المكتبات المطلوبة

استيراد جميع المكتبات المطلوبة لبناء نظام التنبؤ بالمبيعات.

### المكتبات الأساسية:
- pandas & numpy: لمعالجة وتحليل البيانات
- matplotlib & seaborn: لإنشاء الرسوم البيانية
- scikit-learn: لمكتبات التعلم الآلي
- XGBoost: لنماذج التعلم الآلي المتقدمة

### الإعدادات:
- إخفاء التحذيرات للحصول على مخرجات نظيفة
- إعداد الخطوط العربية للرسوم البيانية


In [ ]:
# استيراد المكتبات المطلوبة لمعالجة البيانات والتصور والتعلم الآلي

# مكتبات معالجة البيانات والتحليل
import pandas as pd
import numpy as np

# مكتبات التصور والرسوم البيانية
import matplotlib.pyplot as plt
import seaborn as sns

# مكتبات التعلم الآلي
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

# إخفاء التحذيرات للحصول على مخرجات نظيفة
import warnings
warnings.filterwarnings('ignore')

# إعداد الخطوط العربية للرسوم البيانية
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("تم استيراد جميع المكتبات بنجاح!")
print("جاهز لبدء تحليل البيانات وبناء النموذج")


## الخطوة 2: تحميل واستكشاف البيانات

تحميل بيانات المبيعات اليومية واستكشافها لفهم طبيعتها وخصائصها.

### مصدر البيانات:
- الملف: Daily_sales.csv
- المحتوى: بيانات المبيعات اليومية من 2023 إلى 2024
- الأعمدة: تاريخ المبيعات، يوم الأسبوع، عدد الفواتير، الكمية، الخصم، إجمالي المبيعات

### ما سنتعلمه:
- هيكل البيانات: عدد الصفوف والأعمدة
- أنواع البيانات: الأعمدة الرقمية والنصية
- الإحصائيات الوصفية: المتوسط، الوسيط، الانحراف المعياري
- القيم المفقودة: هل توجد بيانات ناقصة؟


In [ ]:
# تحميل بيانات المبيعات اليومية
print("جاري تحميل البيانات...")
df = pd.read_csv('Daily_sales.csv')

# عرض أول 5 صفوف من البيانات
print("\nأول 5 صفوف من البيانات:")
display(df.head())

# عرض معلومات عن هيكل البيانات
print("\nمعلومات عن البيانات:")
df.info()

# عرض الإحصائيات الوصفية للأعمدة الرقمية
print("\nالإحصائيات الوصفية:")
display(df.describe())

# فحص القيم المفقودة
print("\nالقيم المفقودة:")
missing_values = df.isnull().sum()
display(missing_values)

# عرض نطاق التواريخ
print("\nنطاق التواريخ:")
print(f"من: {df['sale_date'].min()}")
print(f"إلى: {df['sale_date'].max()}")
print(f"إجمالي الأيام: {len(df)} يوم")

print("\nتم تحميل البيانات بنجاح!")


## الخطوة 3: إنشاء الميزات الزمنية

تحويل عمود التاريخ إلى ميزات زمنية مفيدة للنموذج.

### الميزات الزمنية التي سننشئها:

#### الميزات الأساسية:
- year: السنة (2023, 2024)
- month: الشهر (1-12)
- day: اليوم من الشهر (1-31)
- day_of_week: يوم الأسبوع (0=الاثنين, 6=الأحد)
- day_of_year: اليوم من السنة (1-365)
- week_of_year: الأسبوع من السنة (1-52)

#### الميزات المنطقية:
- is_weekend: هل هو عطلة نهاية أسبوع؟ (0 أو 1)
- is_month_start: هل هو أول يوم في الشهر؟ (0 أو 1)
- is_month_end: هل هو آخر يوم في الشهر؟ (0 أو 1)

### لماذا هذه الميزات مهمة؟
- المبيعات في عطلة نهاية الأسبوع قد تكون مختلفة عن أيام العمل
- بداية ونهاية الشهر قد تشهد أنماط مبيعات مختلفة
- الأشهر المختلفة قد تحتوي على مواسم مبيعات مختلفة


In [ ]:
# تحويل عمود التاريخ إلى نوع datetime
print("جاري تحويل التاريخ...")
df['sale_date'] = pd.to_datetime(df['sale_date'])

# ترتيب البيانات حسب التاريخ وإعادة تعيين الفهرس
print("ترتيب البيانات حسب التاريخ...")
df = df.sort_values('sale_date').reset_index(drop=True)

# تعيين عمود التاريخ كفهرس للبيانات
df.set_index('sale_date', inplace=True)

# إنشاء الميزات الزمنية الأساسية
print("إنشاء الميزات الزمنية...")

# الميزات الأساسية
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['day_of_week'] = df.index.dayofweek
df['day_of_year'] = df.index.dayofyear
df['week_of_year'] = df.index.isocalendar().week

# الميزات المنطقية
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_month_start'] = df.index.is_month_start.astype(int)
df['is_month_end'] = df.index.is_month_end.astype(int)

# عرض أول 5 صفوف مع الميزات الجديدة
print("\nأول 5 صفوف مع الميزات الزمنية:")
display(df.head())

# عرض معلومات عن البيانات المحدثة
print("\nمعلومات عن البيانات المحدثة:")
print(f"عدد الأعمدة: {len(df.columns)}")
print(f"عدد الصفوف: {len(df)}")
print(f"الأعمدة الجديدة: {list(df.columns)}")

print("\nتم إنشاء الميزات الزمنية بنجاح!")


## الخطوة 4: إنشاء ميزات التأخير (Lag Features)

إنشاء ميزات التأخير التي تستخدم قيم الأيام السابقة للتنبؤ باليوم الحالي.

### ما هي ميزات التأخير؟
ميزات التأخير هي قيم من الأيام السابقة تُستخدم كمدخلات للتنبؤ باليوم الحالي.

### الميزات التي سننشئها:

#### ميزات تأخير المبيعات:
- sales_lag_1: مبيعات الأمس (1 يوم)
- sales_lag_2: مبيعات قبل أمس (2 يوم)
- sales_lag_3: مبيعات قبل 3 أيام
- sales_lag_7: مبيعات الأسبوع الماضي (7 أيام)
- sales_lag_14: مبيعات قبل أسبوعين (14 يوم)
- sales_lag_30: مبيعات الشهر الماضي (30 يوم)

#### ميزات تأخير أخرى:
- quantity_lag_1, quantity_lag_7: كمية الأمس والأسبوع الماضي
- invoices_lag_1, invoices_lag_7: عدد فواتير الأمس والأسبوع الماضي
- discount_lag_1, discount_lag_7: خصم الأمس والأسبوع الماضي

### لماذا ميزات التأخير مهمة؟
- المبيعات اليوم غالباً ما تتأثر بالمبيعات السابقة
- الأنماط الأسبوعية يمكن اكتشافها من خلال lag_7
- الأنماط الشهرية يمكن اكتشافها من خلال lag_30


In [ ]:
# إنشاء ميزات التأخير للمبيعات
print("إنشاء ميزات تأخير المبيعات...")
for lag in [1, 2, 3, 7, 14, 30]:
    df[f'sales_lag_{lag}'] = df['total_amount'].shift(lag)
    print(f"  تم إنشاء sales_lag_{lag}")

# إنشاء ميزات التأخير للكميات
print("\nإنشاء ميزات تأخير الكميات...")
for lag in [1, 7]:
    df[f'quantity_lag_{lag}'] = df['total_quantity'].shift(lag)
    print(f"  تم إنشاء quantity_lag_{lag}")

# إنشاء ميزات التأخير للفواتير
print("\nإنشاء ميزات تأخير الفواتير...")
for lag in [1, 7]:
    df[f'invoices_lag_{lag}'] = df['invoices_count'].shift(lag)
    print(f"  تم إنشاء invoices_lag_{lag}")

# إنشاء ميزات التأخير للخصومات
print("\nإنشاء ميزات تأخير الخصومات...")
for lag in [1, 7]:
    df[f'discount_lag_{lag}'] = df['total_discount'].shift(lag)
    print(f"  تم إنشاء discount_lag_{lag}")

# عرض أول 5 صفوف مع الميزات الجديدة
print("\nأول 5 صفوف مع ميزات التأخير:")
display(df.head())

# عرض معلومات عن البيانات المحدثة
print("\nمعلومات عن البيانات المحدثة:")
print(f"عدد الأعمدة: {len(df.columns)}")
print(f"عدد الصفوف: {len(df)}")

# فحص القيم المفقودة الناتجة عن ميزات التأخير
print("\nالقيم المفقودة الناتجة عن ميزات التأخير:")
missing_after_lag = df.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_after_lag.max()}")
print(f"العمود مع أكبر عدد قيم مفقودة: {missing_after_lag.idxmax()}")

print("\nتم إنشاء ميزات التأخير بنجاح!")


## الخطوة 5: إنشاء الميزات المتحركة (Rolling Features)

إنشاء الميزات المتحركة التي تحسب الإحصائيات على نافذة زمنية متحركة.

### ما هي الميزات المتحركة؟
الميزات المتحركة تحسب إحصائيات على نافذة زمنية متحركة.

### الميزات التي سننشئها:

#### ميزات متحركة للمبيعات (نوافذ: 7، 14، 30 يوم):
- rolling_mean_sales_X: متوسط المبيعات
- rolling_std_sales_X: الانحراف المعياري للمبيعات
- rolling_max_sales_X: أعلى مبيعات
- rolling_min_sales_X: أقل مبيعات

#### ميزات متحركة للكميات والفواتير (نوافذ: 7، 14 يوم):
- rolling_mean_quantity_X: متوسط الكمية
- rolling_std_quantity_X: الانحراف المعياري للكمية
- rolling_mean_invoices_X: متوسط عدد الفواتير
- rolling_std_invoices_X: الانحراف المعياري لعدد الفواتير

### لماذا الميزات المتحركة مهمة؟
- المتوسط المتحرك يوضح الاتجاه العام
- الانحراف المعياري يوضح التقلبات
- القيم القصوى والدنيا توضح نطاق التقلبات


In [ ]:
# إنشاء الميزات المتحركة للمبيعات
print("إنشاء الميزات المتحركة للمبيعات...")
for window in [7, 14, 30]:
    df[f'rolling_mean_sales_{window}'] = df['total_amount'].rolling(window=window).mean()
    df[f'rolling_std_sales_{window}'] = df['total_amount'].rolling(window=window).std()
    df[f'rolling_max_sales_{window}'] = df['total_amount'].rolling(window=window).max()
    df[f'rolling_min_sales_{window}'] = df['total_amount'].rolling(window=window).min()
    print(f"  تم إنشاء ميزات متحركة للمبيعات (نافذة {window} يوم)")

# إنشاء الميزات المتحركة للكميات
print("\nإنشاء الميزات المتحركة للكميات...")
for window in [7, 14]:
    df[f'rolling_mean_quantity_{window}'] = df['total_quantity'].rolling(window=window).mean()
    df[f'rolling_std_quantity_{window}'] = df['total_quantity'].rolling(window=window).std()
    print(f"  تم إنشاء ميزات متحركة للكميات (نافذة {window} يوم)")

# إنشاء الميزات المتحركة للفواتير
print("\nإنشاء الميزات المتحركة للفواتير...")
for window in [7, 14]:
    df[f'rolling_mean_invoices_{window}'] = df['invoices_count'].rolling(window=window).mean()
    df[f'rolling_std_invoices_{window}'] = df['invoices_count'].rolling(window=window).std()
    print(f"  تم إنشاء ميزات متحركة للفواتير (نافذة {window} يوم)")

# عرض أول 5 صفوف مع الميزات الجديدة
print("\nأول 5 صفوف مع الميزات المتحركة:")
display(df.head())

# عرض معلومات عن البيانات المحدثة
print("\nمعلومات عن البيانات المحدثة:")
print(f"عدد الأعمدة: {len(df.columns)}")
print(f"عدد الصفوف: {len(df)}")

# فحص القيم المفقودة الناتجة عن الميزات المتحركة
print("\nالقيم المفقودة الناتجة عن الميزات المتحركة:")
missing_after_rolling = df.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_after_rolling.max()}")
print(f"العمود مع أكبر عدد قيم مفقودة: {missing_after_rolling.idxmax()}")

print("\nتم إنشاء الميزات المتحركة بنجاح!")


## الخطوة 6: إنشاء الميزات المتقدمة

إنشاء ميزات متقدمة تجمع بين المعلومات الزمنية والإحصائية لتحسين دقة النموذج.

### الميزات المتقدمة التي سننشئها:

#### الميزات الإحصائية:
1. weekly_avg_sales: متوسط المبيعات الأسبوعي (آخر 7 أيام)
2. sales_change_pct: نسبة التغيير في المبيعات مقارنة باليوم السابق
3. monthly_avg_sales: متوسط المبيعات الشهري (نفس الشهر من جميع السنوات)
4. day_of_week_avg: متوسط المبيعات ليوم الأسبوع (نفس يوم الأسبوع من جميع الأسابيع)

### لماذا هذه الميزات مهمة؟

#### فهم الأنماط الموسمية:
- المتوسط الأسبوعي يوضح الاتجاه قصير المدى
- نسبة التغيير تظهر الاتجاه (زيادة أم نقصان)
- المتوسط الشهري يوضح الأنماط الموسمية
- متوسط يوم الأسبوع يوضح الأنماط الأسبوعية

#### تحسين الدقة:
- الميزات الإحصائية توفر سياق إضافي
- الأنماط الموسمية تساعد في التنبؤ
- الاتجاهات قصيرة المدى يمكن توقعها


In [ ]:
# إنشاء الميزات المتقدمة
print("إنشاء الميزات المتقدمة...")

# متوسط المبيعات الأسبوعي (آخر 7 أيام)
df['weekly_avg_sales'] = df['total_amount'].rolling(window=7).mean()
print("  تم إنشاء weekly_avg_sales")

# نسبة التغيير في المبيعات مقارنة باليوم السابق
df['sales_change_pct'] = df['total_amount'].pct_change()
print("  تم إنشاء sales_change_pct")

# متوسط المبيعات الشهري (نفس الشهر من جميع السنوات)
df['monthly_avg_sales'] = df.groupby(df.index.month)['total_amount'].transform('mean')
print("  تم إنشاء monthly_avg_sales")

# متوسط المبيعات ليوم الأسبوع (نفس يوم الأسبوع من جميع الأسابيع)
df['day_of_week_avg'] = df.groupby(df.index.dayofweek)['total_amount'].transform('mean')
print("  تم إنشاء day_of_week_avg")

# عرض أول 5 صفوف مع الميزات الجديدة
print("\nأول 5 صفوف مع الميزات المتقدمة:")
display(df.head())

# عرض معلومات عن البيانات المحدثة
print("\nمعلومات عن البيانات المحدثة:")
print(f"عدد الأعمدة: {len(df.columns)}")
print(f"عدد الصفوف: {len(df)}")

# فحص القيم المفقودة الناتجة عن الميزات المتقدمة
print("\nالقيم المفقودة الناتجة عن الميزات المتقدمة:")
missing_after_advanced = df.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_after_advanced.max()}")
print(f"العمود مع أكبر عدد قيم مفقودة: {missing_after_advanced.idxmax()}")

# عرض قائمة بجميع الأعمدة
print("\nقائمة بجميع الأعمدة:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print("\nتم إنشاء الميزات المتقدمة بنجاح!")


## الخطوة 7: تنظيف البيانات ومعالجة القيم المفقودة

تنظيف البيانات وحذف الصفوف التي تحتوي على قيم مفقودة (NaN) لضمان جودة البيانات قبل التدريب.

### لماذا توجد قيم مفقودة؟

#### ميزات التأخير:
- sales_lag_30 يحتاج 30 يوم من البيانات السابقة
- الصفوف الأولى لا تملك بيانات كافية

#### الميزات المتحركة:
- rolling_mean_30 يحتاج 30 يوم من البيانات
- الصفوف الأولى لا تملك بيانات كافية للنافذة

#### الميزات المتقدمة:
- weekly_avg_sales يحتاج 7 أيام من البيانات
- sales_change_pct يحتاج يوم سابق للمقارنة

### استراتيجية التنظيف:

#### ما سنفعله:
1. حساب عدد الصفوف قبل التنظيف
2. حذف الصفوف التي تحتوي على قيم مفقودة
3. حساب عدد الصفوف بعد التنظيف
4. عرض الإحصائيات عن التغيير

#### تحذير مهم:
- حذف الصفوف يعني فقدان بعض البيانات
- لكن هذا ضروري لضمان جودة النموذج
- البيانات المتبقية ستكون كافية للتدريب


In [ ]:
# تنظيف البيانات وحذف الصفوف التي تحتوي على قيم مفقودة
print("بدء تنظيف البيانات...")

# حساب عدد الصفوف قبل التنظيف
initial_row_count = len(df)
print(f"عدد الصفوف قبل التنظيف: {initial_row_count}")

# فحص القيم المفقودة قبل التنظيف
print("\nالقيم المفقودة قبل التنظيف:")
missing_before = df.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_before.max()}")
print(f"العمود مع أكبر عدد قيم مفقودة: {missing_before.idxmax()}")

# حذف الصفوف التي تحتوي على قيم مفقودة
print("\nحذف الصفوف التي تحتوي على قيم مفقودة...")
df_clean = df.dropna()

# حساب عدد الصفوف بعد التنظيف
final_row_count = len(df_clean)
print(f"عدد الصفوف بعد التنظيف: {final_row_count}")

# حساب عدد الصفوف المحذوفة
rows_removed = initial_row_count - final_row_count
print(f"عدد الصفوف المحذوفة: {rows_removed}")

# فحص القيم المفقودة بعد التنظيف
print("\nالقيم المفقودة بعد التنظيف:")
missing_after = df_clean.isnull().sum()
print(f"أكبر عدد قيم مفقودة: {missing_after.max()}")

# عرض البيانات المنظفة
print("\nأول 5 صفوف من البيانات المنظفة:")
display(df_clean.head())

# عرض معلومات عن البيانات المنظفة
print("\nمعلومات عن البيانات المنظفة:")
print(f"عدد الأعمدة: {len(df_clean.columns)}")
print(f"عدد الصفوف: {len(df_clean)}")
print(f"نطاق التواريخ: من {df_clean.index.min().date()} إلى {df_clean.index.max().date()}")

print("\nتم تنظيف البيانات بنجاح!")


## الخطوة 8: تصور البيانات والرسوم البيانية

إنشاء رسوم بيانية متنوعة لفهم البيانات والأنماط الموجودة فيها.

### الرسوم البيانية التي سننشئها:

#### الرسم البياني الأول: المبيعات اليومية عبر الزمن
- الغرض: فهم الاتجاه العام والأنماط الموسمية
- المحور X: التاريخ
- المحور Y: إجمالي المبيعات
- الفوائد: اكتشاف الاتجاهات طويلة المدى والتقلبات

#### الرسم البياني الثاني: توزيع المبيعات اليومية
- الغرض: فهم توزيع قيم المبيعات
- المحور X: نطاقات المبيعات
- المحور Y: التكرار (عدد الأيام)
- الفوائد: فهم القيم النموذجية والشاذة

#### الرسم البياني الثالث: متوسط المبيعات حسب يوم الأسبوع
- الغرض: فهم الأنماط الأسبوعية
- المحور X: أيام الأسبوع
- المحور Y: متوسط المبيعات
- الفوائد: اكتشاف أيام الذروة والانخفاض

#### الرسم البياني الرابع: متوسط المبيعات حسب الشهر
- الغرض: فهم الأنماط الموسمية
- المحور X: الأشهر
- المحور Y: متوسط المبيعات
- الفوائد: اكتشاف المواسم عالية ومنخفضة المبيعات

### لماذا التصور مهم؟
- اكتشاف الأنماط والاتجاهات في البيانات
- فهم توزيع القيم والعلاقات بين المتغيرات
- تحسين النموذج بناءً على الأنماط المكتشفة


In [ ]:
# إنشاء الرسوم البيانية لتصور البيانات
print("إنشاء الرسوم البيانية...")

# إنشاء شكل الرسم البياني
plt.figure(figsize=(15, 10))

# الرسم البياني الأول: المبيعات اليومية عبر الزمن
plt.subplot(2, 2, 1)
plt.plot(df_clean.index, df_clean['total_amount'], linewidth=1, color='blue')
plt.title('المبيعات اليومية عبر الزمن', fontsize=14, fontweight='bold')
plt.xlabel('التاريخ')
plt.ylabel('إجمالي المبيعات')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# الرسم البياني الثاني: توزيع المبيعات اليومية
plt.subplot(2, 2, 2)
plt.hist(df_clean['total_amount'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('توزيع المبيعات اليومية', fontsize=14, fontweight='bold')
plt.xlabel('إجمالي المبيعات')
plt.ylabel('التكرار')
plt.grid(True, alpha=0.3)

# الرسم البياني الثالث: متوسط المبيعات حسب يوم الأسبوع
plt.subplot(2, 2, 3)
day_names = ['الاثنين', 'الثلاثاء', 'الأربعاء', 'الخميس', 'الجمعة', 'السبت', 'الأحد']
df_clean.groupby('day_of_week')['total_amount'].mean().plot(kind='bar', color='lightcoral')
plt.title('متوسط المبيعات حسب يوم الأسبوع', fontsize=14, fontweight='bold')
plt.xlabel('يوم الأسبوع')
plt.ylabel('متوسط المبيعات')
plt.xticks(range(7), day_names, rotation=45)
plt.grid(True, alpha=0.3)

# الرسم البياني الرابع: متوسط المبيعات حسب الشهر
plt.subplot(2, 2, 4)
month_names = ['يناير', 'فبراير', 'مارس', 'أبريل', 'مايو', 'يونيو',
               'يوليو', 'أغسطس', 'سبتمبر', 'أكتوبر', 'نوفمبر', 'ديسمبر']
df_clean.groupby('month')['total_amount'].mean().plot(kind='bar', color='lightgreen')
plt.title('متوسط المبيعات حسب الشهر', fontsize=14, fontweight='bold')
plt.xlabel('الشهر')
plt.ylabel('متوسط المبيعات')
plt.xticks(range(12), month_names, rotation=45)
plt.grid(True, alpha=0.3)

# تحسين التخطيط وحفظ الرسم
plt.tight_layout()
plt.savefig('time_series_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("تم إنشاء الرسوم البيانية بنجاح!")
print("تم حفظ الرسم كصورة: time_series_analysis.png")


## الخطوة 9: حفظ البيانات المعالجة

حفظ البيانات المعالجة في ملف CSV لاستخدامها لاحقاً في التدريب والتقييم.

### الملف الذي سنحفظه:
- اسم الملف: processed_sales_data.csv
- المحتوى: البيانات المنظفة مع جميع الميزات
- الغرض: استخدامها في التدريب والتقييم

### لماذا نحفظ البيانات المعالجة؟

#### توفير الوقت:
- تجنب إعادة المعالجة في كل مرة
- البدء مباشرة من التدريب
- تسريع العملية بشكل كبير

#### ضمان الاتساق:
- نفس البيانات في كل مرة
- تجنب الأخطاء في المعالجة
- نتائج قابلة للتكرار

#### سهولة الاستخدام:
- تحميل سريع للبيانات
- استخدام في مشاريع أخرى
- مشاركة مع الآخرين

### ما سيتم حفظه:
- جميع الميزات التي أنشأناها
- البيانات المنظفة بدون قيم مفقودة
- الفهرس الزمني (التواريخ)
- 43 عمود من البيانات
